<h1>Batch processing of IP data generated by The University of Michigan</h1>
<h2>Introduction</h2>
<p align="justify">The University of Michigan scans the	Internet IP space on a daily basis using a variety of tools, including zMap tool. The full scan offers a wide range of data about each IP, including data on open ports, HTTP headers, SSL certificates, Operating System, and specific vulnerabilities such as heartbleed. Following analaysis is based on a sampled version of dataset corresponding to 15/07/2017.</p>

<p align="justify">Data is stored in a file, where each line is a JSON. Each line has some constant fields (e.g. {‘ip’:’182.12.23.22’}), the data on each line depends on the ports that were found open on that IP address. </p>

The dataset is currently stored at AWS S3, and available for download from here: <br/>
https://s3-eu-west-1.amazonaws.com/clo34/ip-20170715-sampled.json

More information about IPScan data and zMap can be found using following links. 
<br/>http://scans.io
<br/>https://zmap.io/
<br/>

<p>Architectures used: An Apache Spark cluster running on AWS EMR using Yarn as resource manager.<br/>
Programming language used: Scala </p>

<h2>Analysis</h2>

In [10]:
import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SQLContext

import java.text.DecimalFormat

In [5]:
// Creating an org.apache.spark.sql.SparkSession object.
val session = SparkSession.builder.appName("IPScanData").getOrCreate()

In [6]:
// Creating an org.apache.spark.sql.Dataset object 
val ipDf = session.read.json("s3a://clo34/ip-20170715-sampled.json")

<p>Following self exploratory schema was generated.</p>

In [88]:
ipDf.printSchema()

root
 |-- autonomous_system: struct (nullable = true)
 |    |-- asn: long (nullable = true)
 |    |-- country_code: string (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- organization: string (nullable = true)
 |    |-- path: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- routed_prefix: string (nullable = true)
 |-- ip: string (nullable = true)
 |-- ipint: long (nullable = true)
 |-- location: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- continent: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- country_code: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- postal_code: string (nullable = true)
 |    |-- province: string (nullable = true)
 |    |-- registered_country: string (nullable = true)
 |    |-- registered_country_code: string (nullable 

ipDf = [autonomous_system: struct<asn: bigint, country_code: string ... 5 more fields>, ip: string ... 23 more fields]


[autonomous_system: struct<asn: bigint, country_code: string ... 5 more fields>, ip: string ... 23 more fields]

Following JSON objects within each line were selected for further analysis.
<br/>
<table align="left">
    <tr><th>JSON element</th><th>Data description</th></tr>
    <tr><td>location</td><td>Location of IP</td></tr>
    <tr><td>p443</td><td>Default Port used for HTTPS</td></tr>
    <tr><td>p80</td><td>Default port used for HTTP</td></tr>
    <tr><td>p22</td><td>Default port used for SSH servers</td></tr>
    <tr><td>metadata</td><td>Meta data such as Operating system running on server</td></tr>
</table>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<p>Following fields in above JSON objects were further selected.</p>
<table align="left">
    <tr><th>Field</th><th>Data description</th></tr>
    <tr><td>location.country</td><td>Country in which IP is located</td></tr>
    <tr><td>location.country_code</td><td>Country code of country in which IP is located</td></tr>
    <tr><td>p443.https.heartbleed.heartbleed_vulnerable</td><td>Specifies whether a HTTPS server has heartbleed bug</td></tr>
    <tr><td>metadata.os</td><td>Operating system running on server</td></tr>
    <tr><td>p443.https.tls.cipher_suite.name</td><td>Name of cipher suite used for HTTPS on port 443</td></tr>
    <tr><td>p22.ssh.banner.software_version</td><td>SSH server software version</td></tr>
</table>
<br/>

In [3]:
// Creating an org.apache.spark.sql.Dataset object with selected data elements
val ipDfSelected = ipDf.select("location","p443","p80","p22","metadata")
ipDfSelected.persist()

[location: struct<city: string, continent: string ... 9 more fields>, p443: struct<https: struct<dhe: struct<dh_params: struct<generator: struct<length: bigint, value: string>, prime: struct<length: bigint, value: string>>, support: boolean ... 1 more field>, dhe_export: struct<dh_params: struct<generator: struct<length: bigint, value: string>, prime: struct<length: bigint, value: string>>, support: boolean ... 1 more field> ... 5 more fields>> ... 3 more fields]

<p><b>Finding top ten countries with highest number of IP address records per each country code:</b></p>

In [4]:
/*  
    Dropping entries with null values and filtering to find out non empty values for location.country
    Then grouping entries by location.country_code and location.country 
*/
val cleanedCountryDf = ipDfSelected.na.drop(Seq("location.country")).filter(! col("location.country").like(" "))
val recordsPerCountryDf = cleanedCountryDf.groupBy("location.country_code","location.country").count().
orderBy(desc("count")).drop("country_code").limit(10)

In [18]:
recordsPerCountryDf.show()

+-----------------+-----+
|          country|count|
+-----------------+-----+
|    United States|55048|
|            China|11158|
|           Mexico| 8857|
|          Germany| 6368|
|   United Kingdom| 4676|
|Republic of Korea| 4422|
|           Russia| 3893|
|            Italy| 3795|
|           Brazil| 3577|
|            India| 3466|
+-----------------+-----+



<p><b>Finding relative percentages of HTTP to HTTPS: <br/>
    <table align="left">
    <tr><td>(number of servers that offer HTTP / number of servers that offer HTTP or HTTPS)*100</td></tr>
    <tr><td>(number of servers that offer HTTPS / number of servers that offer HTTP or HTTPS)*100</td></tr>
    <tr><td>(number of servers that offer both HTTP and HTTS / number of servers that offer HTTP or HTTPS)*100</td></tr>
</table>
</b></p>

<p>Servers were assumed to have used only defualt ports for HTTP and HTTPS: 80 and 443 respectively.</p>

In [29]:
val percentages = new DecimalFormat("#.##")

// dropping entries with null values and counting the number of entries for each case. 
val httpIntersectHttpsCount = ipDfSelected.na.drop(Seq("p80","p443")).count() 
val httpCount = ipDfSelected.na.drop(Seq("p80")).count() 
val httpsCount = ipDfSelected.na.drop(Seq("p443")).count()
val httpUnionHttpsCount = http + https - httpIntersectHttpsCount

// calculating required percentages
val httpPercentage = percentages.format(httpCount*100f/httpUnionHttpsCount)
val httpsPercentage = percentages.format(httpsCount*100f/httpUnionHttpsCount)
val bothPercentage = percentages.format(httpIntersectHttpsCount*100f/httpUnionHttpsCount) 

// printing
println(s"httpPercentage = ${httpPercentage}")
println(s"httpsPercentage = ${httpsPercentage}")
println(s"bothPercentage = ${bothPercentage}")

httpPercentage = 51.29%
httpsPercentage = 78.35%
bothPercentage = 29.64%


<p><b>Finding percentage of HTTPS servers having the heartbleed	bug:</b></p>

In [28]:
// dropping the entries with null values and filtering based on condition heartbleed_vulnerable == true 
val heartbleedDf = ipDfSelected.na.drop(Seq("p443.https.heartbleed.heartbleed_vulnerable")).
filter(col("p443.https.heartbleed.heartbleed_vulnerable")===true) 

// counting the entries, calculating percentage and printing. 
val heartbleedCount = heartbleedDf.count()
val heartBleedPercentage = percentages.format(heartbleedCount*100.0f/httpsCount)
println(s"heartBleedPercentage = ${heartBleedPercentage}%")

heartBleedPercentage = 0.15%


<p><b>Finding top ten countries with highest percentage of HTTPS servers that have the heartbleed bug:</b></p>

In [68]:
heartbleedDf.groupBy("location.country").count().withColumn("Percentage", col("count")*100/heartbleedCount).
drop("count").orderBy(desc("Percentage")).withColumn("Percentage",format_number(col("Percentage"),2)).limit(10).show()

+-------------+----------+
|      country|Percentage|
+-------------+----------+
|United States|     17.74|
|        China|      8.06|
|      Germany|      6.45|
|        Italy|      5.65|
|        Japan|      5.65|
|       France|      4.84|
|       Russia|      4.03|
|        India|      3.23|
|       Israel|      3.23|
|    Australia|      3.23|
+-------------+----------+



<p><b>Calculating percentage of servers that are Windows as well as Ububtu, for IP addresses where the operating system was captured:</b></p>

<p>like method was used because there were several versions of Windows in the dataset and all started with the term "Win". Though such a situation was not noticed for Ubuntu, it was decided to apply same procedure for Ubuntu.</p>

In [90]:
// dropping and filtering
val osDf = ipDfSelected.na.drop(Seq("metadata.os"))
val windowsDf = osDf.filter(col("metadata.os").like("%Win%"))
val ubuntuDf = osDf.filter(col("metadata.os").like("%Ubu%"))

In [31]:
val osCount = osDf.count()
val OSInformationAvailablePercentage = percentages.format(osCount*100f/ipDf.count())
val windowsPercentage = percentages.format(windowsDf.count()*100f/osCount)
val ubuntuPercentage = percentages.format(ubuntuDf.count()*100f/osCount)

println(s"OSInformationAvailablePercentage = ${OSInformationAvailablePercentage}%")
println(s"windowsPercentage = ${windowsPercentage}%")
println(s"ubuntuPercentage = ${ubuntuPercentage}%")

OSInformationAvailablePercentage = 5.51%
windowsPercentage = 24.98%
ubuntuPercentage = 21.14%


<p>Only 5.51% of servers have provided information about their Operating System.</p>

<p><b>Finding the ten most common cipher suites used for HTTPS on port 443:</b></p>

In [69]:
ipDfSelected.na.drop(Seq("p443.https.tls.cipher_suite.name")).groupBy("p443.https.tls.cipher_suite.name").count().
orderBy(desc("count")).show(10, truncate=false)

+---------------------------------------+-----+
|name                                   |count|
+---------------------------------------+-----+
|TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256  |17906|
|TLS_RSA_WITH_AES_256_CBC_SHA           |6053 |
|TLS_RSA_WITH_AES_128_GCM_SHA256        |4523 |
|TLS_ECDHE_RSA_WITH_AES_256_CBC_SHA     |4092 |
|TLS_DHE_RSA_WITH_AES_256_CBC_SHA       |2874 |
|TLS_ECDHE_ECDSA_WITH_AES_128_GCM_SHA256|1205 |
|TLS_RSA_WITH_AES_128_CBC_SHA           |1170 |
|TLS_RSA_WITH_RC4_128_SHA               |1083 |
|TLS_DHE_RSA_WITH_AES_128_GCM_SHA256    |674  |
|TLS_RSA_WITH_RC4_128_MD5               |652  |
+---------------------------------------+-----+
only showing top 10 rows



<p><b>Finding the ten most used SSH server software version:</b></p>

In [70]:
ipDfSelected.na.drop(Seq("p22.ssh.banner.software_version")).groupBy("p22.ssh.banner.software_version").count().
orderBy(desc("count")).show(10)

+----------------+-----+
|software_version|count|
+----------------+-----+
|     OpenSSH_5.3| 3294|
| OpenSSH_6.6.1p1| 1761|
|   OpenSSH_6.6.1| 1729|
|   OpenSSH_7.2p2| 1445|
|   OpenSSH_6.7p1| 1037|
|   dropbear_0.46|  649|
|   OpenSSH_6.0p1|  565|
|          ROSSSH|  533|
|     OpenSSH_4.3|  465|
|      Cisco-1.25|  436|
+----------------+-----+
only showing top 10 rows



In [9]:
%%html
<style> 
h1,h2,p,table,h4{font-family: Helmet, Freesans, Helvetica, Arial, sans-serif; }
h2{padding:0}
p,h4{ font-size: 18px;}
tr{font-size: 16px;}
</style> 